In [1]:
import numpy as np
import pandas as pd
import bz2
import xml.sax
import mwparserfromhell

In [2]:
wiki = 'enwiki-20190101-pages-articles-multistream.xml.bz2'
index = 'enwiki-20190101-pages-articles-multistream-index.txt.bz2'

In [9]:
class WikiXMLHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []
        self._books = []
        
        
    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)
            
            
    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []
            
        
    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))
            # Send the page to the process article function
#             book = process_article(**self._values,
#                                    template = 'Infobox book')
            # If article is a book append to list of books
#             if book:
#                 self._books.append(book)

In [10]:
# Object for handling xml
handler = WikiXMLHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# Iteratively process file
for line in bz2.BZ2File(wiki, 'r'):
    parser.feed(line)
    
    # Stop when 3 articles have been found
    if len(handler._pages) > 2:
        break

In [14]:
handler._pages[0]

('AccessibleComputing',
 '#REDIRECT [[Computer accessibility]] \n \n {{R from move}} \n {{R from CamelCase}} \n {{R unprintworthy}}')

In [20]:
# Create the wiki article
wiki = mwparserfromhell.parse(handler._pages[0][1])

# Find the wikilinks
wikilinks = [x.title for x in wiki.filter_wikilinks()]
wikilinks[:5]

['Computer accessibility']

In [23]:
wiki.strip_code().strip()

'REDIRECT Computer accessibility'

In [ ]:
wiki.filter_templates('Infobox book')